Creating tables in SQL
---------------------

Before we actually get into basic SQL queries (**asking questions _of_ data in tables**), we'll look at some of the basics about how to **create** tables.

In [1]:
%load_ext sql
%sql sqlite://

We will start by creating a SQL database on Books and Authors. Let's first create a table named `Author` with three attributes:
> * `authorid`
> * `firstname`
> * `lastname`

In [2]:
%%sql
CREATE TABLE Author(
    authorid INTEGER PRIMARY KEY,    
    firstname CHAR(20),
    lastname CHAR(30)
);

 * sqlite://
Done.


[]

Let's see how to view the **schema** of existing tables. There are several ways, including but not limited to:
* DESCRIBE tablename
* SHOW CREATE TABLE tablename
* SHOW COLUMNS tablename

Unfortunately, support for these varies widely between DBMSs, and is also limited by our IPython interface (for example sqlite, which we are using, does not support the above; it does have a `.schema tablename` command, however this doesn't work in IPython notebooks...)

One that does work for us here though is:

In [30]:
#Metadata in SQLite can be obtained using the PRAGMA command
%sql PRAGMA table_info(Author);

 * sqlite://
Done.


cid,name,type,notnull,dflt_value,pk
0,authorid,INTEGER,0,None,1
1,firstname,CHAR(20),0,None,0
2,lastname,CHAR(30),0,None,0


And, we can get the exact statement used to create the table as follows:

In [31]:
%sql SELECT sql FROM sqlite_master WHERE name = 'Author';

 * sqlite://
Done.


sql
"CREATE TABLE Author( authorid INTEGER PRIMARY KEY, firstname CHAR(20), lastname CHAR(30) )"


To insert a new tuple in a table, we use the SQL command INSERT.

In [3]:
%sql INSERT INTO Author VALUES(1,'Dan', 'Brown');

 * sqlite://
1 rows affected.


[]

Alternatively, we can also insert a tuple by specifying the corresponding attributes.

In [4]:
%sql INSERT INTO Author(authorid, lastname, firstname) VALUES(3, 'Zhang', 'James');

 * sqlite://
1 rows affected.


[]

To check that the tuples has indeed been added to the table, we can ask:

In [5]:
%%sql 
SELECT *
FROM Author;

 * sqlite://
Done.


authorid,firstname,lastname
1,Dan,Brown
3,James,Zhang


The attribute `authorid` is a *primary key*. This means that it is not possible to add in the table another author with the same value. For example, see what happens when we attempt to violate the constraint.

In [ ]:
%sql INSERT INTO Author VALUES('01', 'Jules', 'Verne');

Foreign Keys
------------------

To make SQLite enforce foreign key constraints, we need to run the following command

In [6]:
%sql PRAGMA foreign_keys = ON;

 * sqlite://
Done.


[]

In [7]:
%%sql
CREATE TABLE Book(
  bookid INTEGER PRIMARY KEY,
  title TEXT,
  authorid INTEGER,
  FOREIGN KEY (authorid) REFERENCES Author(authorid)
);

 * sqlite://
Done.


[]

In [15]:
%%sql
SELECT *
FROM Book

 * sqlite://
Done.


bookid,title,authorid
5634,Da Vinvi Code,4


In [9]:
%sql INSERT INTO Book VALUES(5634, 'Da Vinvi Code', 001);

 * sqlite://
1 rows affected.


[]

If however we try to insert a tuple that has an `authorid` not in the table `Author`, then it fails.

In [10]:
%sql INSERT INTO Book VALUES(5674, 'Enders Game', 003);

 * sqlite://
1 rows affected.


[]

The same problem will occur if we try to delete from `Author` a tuple that is referred to from the table `Book`.

In [ ]:
%sql DELETE FROM Author WHERE authorid = 1 ;

 * sqlite://
0 rows affected.


[]

Or if we try to update the value of the tuple.

In [ ]:
%%sql
UPDATE Author
SET authorid = 4
WHERE authorid = 1 ;

The *default* behavior of SQL is to reject actions (insertions, deletions, updates) that violate the foreign key constraints. We can change this behavior by changing the table definition.

In [13]:
%%sql
DROP Table Book;
CREATE TABLE Book(
  bookid INTEGER PRIMARY KEY,
  title TEXT,
  authorid INTEGER,
  FOREIGN KEY (authorid) REFERENCES Author(authorid)
    ON UPDATE CASCADE
    ON DELETE CASCADE
);

INSERT INTO Book VALUES(5634, 'Da Vinvi Code', 001);

 * sqlite://
Done.
Done.
1 rows affected.


[]

Now let's try again!

In [16]:
%%sql 
UPDATE Author
SET authorid = 5
WHERE authorid = 1 ;

 * sqlite://
0 rows affected.


[]

In [17]:
%sql SELECT * from Book ;

 * sqlite://
Done.


bookid,title,authorid
5634,Da Vinvi Code,4
